In [ ]:
# Fill in student ID and name
# 
student_id = "s224529554"
student_first_last_name = "Panha Ath"
print(student_id, student_first_last_name)

s224529554 Panha Ath


In [ ]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import dash

In [ ]:

df = pd.read_csv("gyroscope_data.csv")

df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms", errors="coerce")
if df["timestamp"].isna().all():
    df["timestamp"] = pd.to_datetime(np.arange(len(df)), unit="ms")


VARIABLES = ["x", "y", "z"]

GRAPH_TYPES = {
    "Line Chart": "line",
    "Scatter Plot": "scatter",
    "Histogram": "histogram",
    "Box Plot": "box",
    "Violin Plot": "violin",
}


app = Dash(__name__)

app.layout = html.Div(
    [
        html.H1("Gyroscope Data Visualization", style={"textAlign": "center"}),

        html.Div(
            [
                html.Div(
                    [
                        html.Label("Select Graph Type:"),
                        dcc.Dropdown(
                            id="graph-type-dropdown",
                            options=[{"label": k, "value": v} for k, v in GRAPH_TYPES.items()],
                            value="line",
                            clearable=False,
                        ),
                    ],
                    style={"width": "30%", "display": "inline-block", "padding": "10px"},
                ),
                html.Div(
                    [
                        html.Label("Select Variables:"),
                        dcc.Dropdown(
                            id="variable-dropdown",
                            options=[{"label": "All", "value": "all"}]
                            + [{"label": v.upper(), "value": v} for v in VARIABLES],
                            value=["all"],  # multi-select
                            multi=True,
                            clearable=False,
                        ),
                    ],
                    style={"width": "30%", "display": "inline-block", "padding": "10px"},
                ),

                html.Div(
                    [
                        html.Label("X-axis:"),
                        dcc.RadioItems(
                            id="xaxis-mode",
                            options=[
                                {"label": "Auto",      "value": "auto"},
                                {"label": "Time",      "value": "time"},
                                {"label": "Sample #",  "value": "index"},
                            ],
                            value="auto",
                            inline=True,
                        ),
                    ],
                    style={"width": "24%", "display": "inline-block", "padding": "10px", "verticalAlign": "top"},
                ),
                
                html.Div(
                    [
                        html.Label("Samples to Display:"),
                        dcc.Input(
                            id="sample-size-input",
                            type="number",
                            value=min(100, len(df)),
                            min=10,
                            max=len(df),
                            step=10,
                        ),
                        html.Button("Previous", id="prev-button", n_clicks=0, style={"margin": "5px"}),
                        html.Button("Next", id="next-button", n_clicks=0, style={"margin": "5px"}),
                    ],
                    style={"width": "30%", "display": "inline-block", "padding": "10px"},
                ),
            ],
            style={"border": "1px solid #ddd", "borderRadius": "5px", "padding": "10px", "margin": "10px"},
        ),

        
        dcc.Graph(id="gyroscope-graph"),

       
        html.Div(
            [
                html.H3("Data Summary (Current Window)"),
                dash_table.DataTable(
                    id="summary-table",
                    style_cell={"textAlign": "left"},
                    style_header={"backgroundColor": "lightgrey", "fontWeight": "bold"},
                ),
                html.Div(id="range-info", style={"marginTop": "8px", "fontStyle": "italic"}),
            ],
            style={"margin": "20px"},
        ),

       
        dcc.Store(id="start-idx", data=0),
        dcc.Store(id="prev-clicks-store", data=0),
        dcc.Store(id="next-clicks-store", data=0),
    ]
)


def resolve_vars(selected):
    if not selected:
        return VARIABLES
    if isinstance(selected, list) and "all" in selected:
        return VARIABLES
    if selected == "all":
        return VARIABLES
    return selected if isinstance(selected, list) else [selected]

def clamp_window(total, n, start):
    n = max(1, int(n)) if n else total
    start = max(0, min(start, max(0, total - n)))
    end = start + n
    return start, end, n

def with_timestamp_plot(df_in: pd.DataFrame) -> pd.DataFrame:
    out = df_in.copy()
    ts_ms = out["timestamp"].dt.floor("ms")
    dup_ix = ts_ms.groupby(ts_ms).cumcount()
    out["timestamp_plot"] = ts_ms + pd.to_timedelta(dup_ix, unit="us")
    return out

@callback(
    Output("start-idx", "data"),
    Output("prev-clicks-store", "data"),
    Output("next-clicks-store", "data"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    Input("sample-size-input", "value"),
    State("start-idx", "data"),
    State("prev-clicks-store", "data"),
    State("next-clicks-store", "data"),
)
def update_start_index(prev_clicks, next_clicks, n_samples, start_idx, prev_store, next_store):
    prev_clicks = prev_clicks or 0
    next_clicks = next_clicks or 0
    start_idx = start_idx or 0
    prev_store = prev_store or 0
    next_store = next_store or 0

    total = len(df)
    n_samples = max(1, int(n_samples)) if n_samples else total

   
    if next_clicks > next_store:
        start_idx = min(max(0, total - n_samples), start_idx + n_samples)
    elif prev_clicks > prev_store:
        start_idx = max(0, start_idx - n_samples)
    else:
        
        start_idx = min(start_idx, max(0, total - n_samples))

    return start_idx, prev_clicks, next_clicks

def with_timestamp_plot(df_in):
    out = df_in.copy()
    ts_ms  = out["timestamp"].dt.floor("ms")
    dup_ix = ts_ms.groupby(ts_ms).cumcount()
    out["timestamp_plot"] = ts_ms + pd.to_timedelta(dup_ix, unit="us")
    return out



@callback(
    Output("gyroscope-graph", "figure"),
    Output("summary-table", "data"),
    Output("summary-table", "columns"),
    Output("range-info", "children"),
    Input("graph-type-dropdown", "value"),
    Input("variable-dropdown", "value"),
    Input("sample-size-input", "value"),
    Input("start-idx", "data"),
    Input("xaxis-mode", "value"),
)
def update_graph(graph_type, selected_vars, n_samples, start_idx, xaxis_mode):
    display_vars = resolve_vars(selected_vars)
    total = len(df)
    start_idx, end_idx, n_samples = clamp_window(total, n_samples or total, start_idx or 0)
    df_subset = df.iloc[start_idx:end_idx].copy()
    df_plot = with_timestamp_plot(df_subset)

    if graph_type == "line":
        df_subset = df_subset.copy()

        ts_ms = df_subset["timestamp"].dt.floor("ms")
        dup_ix = ts_ms.groupby(ts_ms).cumcount() 
        df_subset["timestamp_plot"] = ts_ms + pd.to_timedelta(dup_ix, unit="us")

        melted = df_subset.melt(
            id_vars=["timestamp_plot"], 
            value_vars=display_vars,
            var_name="axis", 
            value_name="value")
        fig = px.line(melted, 
                      x="timestamp_plot", 
                      y="value", 
                      color="axis", 
                      title="Raw gyroscope data")
        fig.update_layout(xaxis_title="Time", yaxis_title="Angular velocity")

    elif graph_type == "scatter":
        if len(display_vars) == 2:
            # classic pairwise scatter: axis[0] vs axis[1]
            fig = px.scatter(
                df_subset, x=display_vars[0], y=display_vars[1],
                title=f"{display_vars[0].upper()} vs {display_vars[1].upper()}"
            )
        else:
            # time-scatter when All (or ≥3) are selected: X/Y/Z vs time
            melted = df_plot.melt(
                id_vars=["timestamp_plot"], value_vars=display_vars,
                var_name="axis", value_name="value"
            )
            fig = px.scatter(
                melted, x="timestamp_plot", y="value", color="axis",
                title="Scatter over Time (X/Y/Z)", opacity=0.8
            )
            fig.update_traces(marker=dict(size=6))
            fig.update_layout(xaxis_title="Time", yaxis_title="Value")
    
    elif graph_type == "histogram":
        melted = df_subset.melt(value_vars=display_vars, var_name="axis", value_name="value")
        fig = px.histogram(melted, x="value", color="axis", barmode="overlay", nbins=50,
                           marginal="rug", title="Distribution (Histogram)")
        fig.update_traces(opacity=0.7)
        fig.update_layout(xaxis_title="Value", yaxis_title="Count")
        
    elif graph_type == "box":
        fig = go.Figure()
        for v in display_vars:
            fig.add_trace(go.Box(y=df_subset[v], name=v.upper()))
        fig.update_layout(title="Box Plot", xaxis_title="Variable", yaxis_title="Value")
        
    elif graph_type == "violin":
        fig = go.Figure()
        for v in display_vars:
            fig.add_trace(go.Violin(y=df_subset[v], name=v.upper(), box_visible=True, points="all"))
        fig.update_layout(title="Violin Plot", xaxis_title="Variable", yaxis_title="Value")
    else:
        # fallback
        fig = px.line(df_subset, x="timestamp", y=display_vars[0], title=f"{display_vars[0].upper()} over Time")

    fig.update_layout(margin={"l": 40, "b": 40, "t": 50, "r": 10}, hovermode="closest")

    rows = []
    for v in display_vars:
        s = pd.to_numeric(df_subset[v], errors="coerce")
        rows.append({
            "Variable": v.upper(),
            "Count": int(s.count()),
            "Mean": f"{s.mean():.6f}" if s.count() else None,
            "Std Dev": f"{s.std():.6f}" if s.count() > 1 else None,
            "Min": f"{s.min():.6f}" if s.count() else None,
            "Max": f"{s.max():.6f}" if s.count() else None,
            "Median": f"{s.median():.6f}" if s.count() else None,
        })
    columns = [{"name": c, "id": c} for c in rows[0].keys()] if rows else []

    t0 = df_subset["timestamp"].iloc[0] if len(df_subset) else None
    t1 = df_subset["timestamp"].iloc[-1] if len(df_subset) else None
    range_text = f"Showing samples {start_idx}–{end_idx - 1} of {total}"
    if pd.api.types.is_datetime64_any_dtype(df["timestamp"]):
        range_text += f" | Time range: {t0} → {t1}"

    return fig, rows, columns, range_text

if __name__ == "__main__":
    app.run(debug=True, jupyter_mode = 'tab')


Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

In [ ]:
 html.Div(
                    [
                        html.Label("X-axis:"),
                        dcc.RadioItems(
                            id="xaxis-mode",
                            options=[
                                {"label": "Auto",      "value": "auto"},
                                {"label": "Time",      "value": "time"},
                                {"label": "Sample #",  "value": "index"},
                            ],
                            value="auto",
                            inline=True,
                        ),
                    ],
                    style={"width": "24%", "display": "inline-block", "padding": "10px", "verticalAlign": "top"},
                ),